In [39]:

# importing the required libraries
import pandas as pd
import numpy as np
# Visualisation libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from prophet import Prophet
import plotly.express as px
import plotly.express as px
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.metrics import mean_absolute_percentage_error
from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing


In [40]:
df=pd.read_csv('Code HIV/number-of-new-hiv-infections-per-year.csv')
# df['Year'] = df['Year'].astype('datetime64')
# df['Date'] = df['Year'].apply(lambda x: '-'.join(x.values.astype(str)))
df.fillna(0)
df.dtypes


Entity                                                                object
Code                                                                  object
Year                                                                   int64
Adults (ages 15+) and children (ages 0-14) newly infected with HIV     int64
dtype: object

In [41]:
for i in range(0,31):
    df['Day']=1
    df['Month'] = 1
df['Date']=pd.to_datetime(df[['Year','Month','Day']])
df

,Entity,Code,Year,Adults (ages 15+) and children (ages 0-14) newly infected with HIV,Day,Month,Date
0,Afghanistan,AFG,1990,100,1,1,1990-01-01
1,Afghanistan,AFG,1991,100,1,1,1991-01-01
2,Afghanistan,AFG,1992,100,1,1,1992-01-01
3,Afghanistan,AFG,1993,200,1,1,1993-01-01
4,Afghanistan,AFG,1994,200,1,1,1994-01-01
...,...,...,...,...,...,...,...
4064,Zimbabwe,ZWE,2016,41000,1,1,2016-01-01
4065,Zimbabwe,ZWE,2017,36000,1,1,2017-01-01
4066,Zimbabwe,ZWE,2018,30000,1,1,2018-01-01
4067,Zimbabwe,ZWE,2019,30000,1,1,2019-01-01


In [42]:
df = df[df['Entity'] == 'World']
df_new=df[['Date','Adults (ages 15+) and children (ages 0-14) newly infected with HIV']]

df_new.columns = ['ds', 'y']
print(df_new.to_string())
# df_new.shape

             ds        y
3945 1990-01-01  2000000
3946 1991-01-01  2300000
3947 1992-01-01  2500000
3948 1993-01-01  2700000
3949 1994-01-01  2900000
3950 1995-01-01  3000000
3951 1996-01-01  3000000
3952 1997-01-01  3000000
3953 1998-01-01  3000000
3954 1999-01-01  3000000
3955 2000-01-01  2900000
3956 2001-01-01  2800000
3957 2002-01-01  2700000
3958 2003-01-01  2600000
3959 2004-01-01  2500000
3960 2005-01-01  2400000
3961 2006-01-01  2400000
3962 2007-01-01  2300000
3963 2008-01-01  2200000
3964 2009-01-01  2200000
3965 2010-01-01  2100000
3966 2011-01-01  2100000
3967 2012-01-01  2000000
3968 2013-01-01  1900000
3969 2014-01-01  1800000
3970 2015-01-01  1800000
3971 2016-01-01  1700000
3972 2017-01-01  1700000
3973 2018-01-01  1600000
3974 2019-01-01  1500000
3975 2020-01-01  1500000


In [43]:
df_new['y'] = df_new['y'].cumsum()
df_new

C:\Users\anure\AppData\Local\Temp\ipykernel_2920\3627608061.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ds,y
3945,1990-01-01,2000000
3946,1991-01-01,4300000
3947,1992-01-01,6800000
3948,1993-01-01,9500000
3949,1994-01-01,12400000
3950,1995-01-01,15400000
3951,1996-01-01,18400000
3952,1997-01-01,21400000
3953,1998-01-01,24400000
3954,1999-01-01,27400000


In [25]:
model_compare=df_new.iloc[26:]
error_compare=pd.DataFrame(columns=['Model','RMSE', 'MSE','MAPE','R2 Score'])
model_compare

,ds,y
3971,2016-01-01,65800000
3972,2017-01-01,67500000
3973,2018-01-01,69100000
3974,2019-01-01,70600000
3975,2020-01-01,72100000


In [26]:
df_new["Days Since"]=(df_new['ds']-df_new['ds'].min())
df_new

C:\Users\anure\AppData\Local\Temp\ipykernel_2920\809864701.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ds,y,Days Since
3945,1990-01-01,2000000,0 days
3946,1991-01-01,4300000,365 days
3947,1992-01-01,6800000,730 days
3948,1993-01-01,9500000,1096 days
3949,1994-01-01,12400000,1461 days
3950,1995-01-01,15400000,1826 days
3951,1996-01-01,18400000,2191 days
3952,1997-01-01,21400000,2557 days
3953,1998-01-01,24400000,2922 days
3954,1999-01-01,27400000,3287 days


In [27]:
df_new["Days Since"]=(df_new['ds']-df_new['ds'].min())
df_new

C:\Users\anure\AppData\Local\Temp\ipykernel_2920\809864701.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,ds,y,Days Since
3945,1990-01-01,2000000,0 days
3946,1991-01-01,4300000,365 days
3947,1992-01-01,6800000,730 days
3948,1993-01-01,9500000,1096 days
3949,1994-01-01,12400000,1461 days
3950,1995-01-01,15400000,1826 days
3951,1996-01-01,18400000,2191 days
3952,1997-01-01,21400000,2557 days
3953,1998-01-01,24400000,2922 days
3954,1999-01-01,27400000,3287 days


In [28]:
temp =pd.DataFrame(columns=['Year','Day', 'Month'])
temp

,Year,Day,Month


In [29]:
# for i in range(2021,2025):
temp['Year']=range(1990,2026)
temp['Month']=1
temp['Day']=1
    
temp['Date']=pd.to_datetime(temp[['Year','Month','Day']])
temp.drop(['Year','Day','Month'],axis=1, inplace=True)

temp["Days Since"]=(temp['Date']-temp['Date'].min())

temp

,Date,Days Since
0,1990-01-01,0 days
1,1991-01-01,365 days
2,1992-01-01,730 days
3,1993-01-01,1096 days
4,1994-01-01,1461 days
5,1995-01-01,1826 days
6,1996-01-01,2191 days
7,1997-01-01,2557 days
8,1998-01-01,2922 days
9,1999-01-01,3287 days


In [38]:
df_future= temp[31:]
model_compare=df_future

model_compare

,Date,Days Since
31,2021-01-01,11323 days
32,2022-01-01,11688 days
33,2023-01-01,12053 days
34,2024-01-01,12418 days
35,2025-01-01,12784 days


In [31]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree = 7) 

In [32]:
train_poly=poly.fit_transform(np.array(df_new["Days Since"]).reshape(-1,1))
valid_poly=poly.fit_transform(np.array(df_future["Days Since"]).reshape(-1,1))
y_poly=df_new["y"]
y_poly

3945     2000000
3946     4300000
3947     6800000
3948     9500000
3949    12400000
3950    15400000
3951    18400000
3952    21400000
3953    24400000
3954    27400000
3955    30300000
3956    33100000
3957    35800000
3958    38400000
3959    40900000
3960    43300000
3961    45700000
3962    48000000
3963    50200000
3964    52400000
3965    54500000
3966    56600000
3967    58600000
3968    60500000
3969    62300000
3970    64100000
3971    65800000
3972    67500000
3973    69100000
3974    70600000
3975    72100000
Name: y, dtype: int64

In [33]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso
linreg=LinearRegression(normalize=True)
linreg.fit(train_poly,y_poly)

C:\Users\anure\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning:

'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)





LinearRegression(normalize=True)

In [34]:
prediction_poly=linreg.predict(valid_poly)

In [35]:
# fig=go.Figure()
# fig.add_trace(go.Scatter(x=df_new['ds'], y=df_new["y"],
#                     mode='lines+markers',name="Train Data for Confirmed Cases"))

# fig.add_trace(go.Scatter(x=model_compare['Date'], y=model_compare["Pol"],
#                     mode='lines+markers',name="Future Prediction of Confirmed Cases",))
# fig.update_layout(title="Future Cases Prediction using Holt's Winter Model",
#                  xaxis_title="Date",yaxis_title="Confirmed Cases",legend=dict(x=0,y=1,traceorder="normal"))
# fig.show()

In [36]:
model_compare['Polynomial Regression']= prediction_poly
model_compare

C:\Users\anure\AppData\Local\Temp\ipykernel_2920\4263082300.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Date,Days Since,Polynomial Regression
31,2021-01-01,11323 days,7.371908e+07
32,2022-01-01,11688 days,7.547609e+07
33,2023-01-01,12053 days,7.752909e+07
34,2024-01-01,12418 days,8.006115e+07
35,2025-01-01,12784 days,8.332401e+07


In [37]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=df_new['ds'], y=df_new["y"],
                    mode='lines+markers',name="Train Data for Confirmed Cases"))

fig.add_trace(go.Scatter(x=model_compare['Date'], y=model_compare["Polynomial Regression"],
                    mode='lines+markers',name="Future Prediction of Confirmed Cases",))
fig.update_layout(title="Future Cases Prediction using Polynomial Regression",
                 xaxis_title="Date",yaxis_title="Cumulative Cases",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()